# Project 5: Fitting RNN Model

We chose RNN because it works well with sequential data, and considering the data is timeseries (in which the order of data matters) RNN may prove to be highly well performing. 

In order to use Keras efficiently, this notebook was created and ran through Google Colab.

### Imports and Combined Dataframe CSV Reading

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GRU

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Source: https://stackoverflow.com/questions/48841500/how-to-upload-my-dataset-into-google-colab

from google.colab import files
uploaded = files.upload()

import io
import zipfile

zf = zipfile.ZipFile(io.BytesIO(uploaded['complete_df.csv.zip']))
cdc_county = pd.read_csv(zf.open('complete_df.csv'))

Saving complete_df.csv.zip to complete_df.csv (3).zip


### Reset Index and Data Checking

In [3]:
cdc_county.set_index("date", drop = True, inplace = True)

In [4]:
cdc_county.head()

,County,County only,FIPS code,State,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,stringency_index,government_response_index,containment_health_index,economic_support_index,c1_school_closing,c1_flag,c2_workplace_closing,c2_flag,c3_cancel_public_events,c3_flag,c4_restrictions_on_gatherings,c4_flag,c5_close_public_transport,c5_flag,c6_stay_at_home_requirements,c6_flag,c7_movementrestrictions,c7_flag,c8_internationaltravel,e1_income_support,e1_flag,e2_debtrelief,h1_public_information_campaigns,h1_flag,h2_testing_policy,h3_contact_tracing,h6_facial_coverings,h6_flag,h7_vaccination_policy,h7_flag,...,% ventilators in use by COVID-19 patient - absolute change,% hospital CCNs reporting any utilization data at least once - last 7 days,% hospital CCNs reporting all utilization data at least once - last 7 days,% hospital CCNs ever reporting utilization data,% hospital CCNs reporting COVID-19 admissions at least once - last 7 days,People who are fully vaccinated as % of total population,People who are fully vaccinated as % of population - ages 65+,% Uninsured,% In Poverty,% Over Age 65,Average household size,% Non-Hispanic Black,% Hispanic,% Non-Hispanic Native American / Alaskan Native,% Non-Hispanic Asian,SVI score,CCVI score,Forecasted case trajectory,Viral (RT-PCR) lab test positivity rate - absolute change (may be an underestimate due to delayed reporting).1,Total RT-PCR diagnostic tests - % change (may be an underestimate due to delayed reporting).1,Testing latency - absolute change.1,% tests resulted in 3 or fewer days - absolute change.1,Viral (RT-PCR) lab test positivity rate - 15-21 days ago (may be an underestimate due to delayed reporting),% tests resulted in 3 or fewer days - 15-21 days ago,% Native American / Alaskan Native,% Asian,ihe_plus_5000_fte_cat,rapid_riser_cat,forecast_case_traj_cat,area_of_concern_cat,cases_national_total_change,total_full_vac_%_change,age_65_full_vac_%_change,com_trans_last7_cat,cbsa_type_cat,county_type_cat,Lat,Long_,daily_deaths,daily_confirmed_cases
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-01-02,"Abbeville County, SC",Abbeville County,45001,South Carolina,-17.0,-10.0,-11.0,-1.0,-18.0,8.0,49.07,55.36,57.92,37.5,2.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,1.0,0.0,1.0,2.0,1.0,3.0,2.0,3.0,0.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.116205,0.225468,0.22355,2.51,0.274228,0.01639,0.0,0.0,0.8564,0.698504,Uncertain,0.027,0.06,0.0,0.01,0.072,0.94,0.0,0.003629,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,34.223334,-82.461707,25,1282
2021-01-03,"Abbeville County, SC",Abbeville County,45001,South Carolina,-13.0,-8.0,0.0,17.0,-21.0,6.0,49.07,55.36,57.92,37.5,2.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,1.0,0.0,1.0,2.0,1.0,3.0,2.0,3.0,0.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.116205,0.225468,0.22355,2.51,0.274228,0.01639,0.0,0.0,0.8564,0.698504,Uncertain,-0.007,0.34,0.0,0.01,0.097,0.94,0.0,0.003629,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,34.223334,-82.461707,26,1294
2021-01-04,"Abbeville County, SC",Abbeville County,45001,South Carolina,-8.0,-1.0,-9.0,-10.0,-17.0,7.0,49.07,55.36,57.92,37.5,2.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,1.0,0.0,1.0,2.0,1.0,3.0,2.0,3.0,0.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.116205,0.225468,0.22355,2.51,0.274228,0.01639,0.0,0.0,0.8564,0.698504,Uncertain,-0.021,0.01,0.0,-0.01,0.106,0.96,0.0,0.003629,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,34.223334,-82.461707,26,1301
2021-01-05,"Abbeville County, SC",Abbeville County,45001,South Carolina,-10.0,-2.0,-15.0,-10.0,-15.0,6.0,49.07,55.36,57.92,37.5,2.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,1.0,0.0,1.0,2.0,1.0,3.0,2.0,3.0,0.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.116205,0.225468,0.22355,2.51,0.274228,

In [5]:
pd.DataFrame(cdc_county.isnull().sum()).sort_values(0,ascending=False).head()

,0
County,0
Confirmed COVID-19 admissions per 100 inpatient beds - percent increase,0
People who are fully vaccinated as % of population - ages 65+,0
People who are fully vaccinated as % of total population,0
% hospital CCNs reporting COVID-19 admissions at least once - last 7 days,0


### Feature Selection

Columns are dropped, numified and dummied accordingly, to remove unnecessary features to avoid overfitting and so that the model would accept the features.

In [6]:
cdc_county.drop(columns = ["FIPS code",
                           "month",
                           "State Abbreviation",
                           "Lat",
                           "Long_",
                           "County only"
                          ], inplace = True)

In [7]:
target_col_map = {'Likely Decreasing':0, 'Uncertain': 1, 'Likely Increasing': 2}
cdc_county['Forecasted case trajectory'] = cdc_county['Forecasted case trajectory'].map(target_col_map)
cdc_county['Forecasted case trajectory'].value_counts()

1    304715
0     37466
2       297
Name: Forecasted case trajectory, dtype: int64

In [8]:
obj_cols = [cols for cols in cdc_county.columns if cdc_county[cols].dtypes == "object" and cols != "County" and cols != 'Forecasted case trajectory']

In [9]:
cdc_county = pd.get_dummies(data = cdc_county, columns = obj_cols, drop_first = True)

In [10]:
cdc_county.drop(columns = ["Forecasted case trajectory","County"]).shape

(342478, 186)

### Fitting 3,000+ Models

A for loop was created to ensure that each county is predicted individually and that the data acts as a sequential timeseries (index is unique).

In [11]:
unique_counties = cdc_county["County"].unique()

In [12]:
ls = []

for county in unique_counties:
    dic = {}
    df = cdc_county[cdc_county["County"] == county]
    df.sort_index(ascending = True)
    X = df.drop(columns = ["County","Forecasted case trajectory"])
    y = df['Forecasted case trajectory']
    X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle = False, random_state = 42)
    y_train = to_categorical(y_train, 3)
    y_test = to_categorical(y_test, 3)
    ss = StandardScaler()
    X_train_sc = ss.fit_transform(X_train)
    X_test_sc = ss.transform(X_test)
    train_sequences = TimeseriesGenerator(X_train_sc, y_train, length=3, batch_size=64)
    batch_x, batch_y = train_sequences[0]
    test_sequences = TimeseriesGenerator(X_test_sc, y_test, length=3, batch_size=64)
    model = Sequential()
    model.add(GRU(186, input_shape=train_sequences[0][0][0].shape, return_sequences=True))  # if next layer is GRU
    model.add(GRU(186, return_sequences=False)) 
    model.add(Dense(100, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['acc'])
    history = model.fit(train_sequences, validation_data=test_sequences, epochs=10, verbose=0)
    dic["date"] = list(X_test.index)
    dic["County"] = county
    dic["y_test"] = y_test
    dic["y_preds"] = model.predict(test_sequences[0][0])
    print(county)
    ls.append(dic)
    

Streaming output truncated to the last 5000 lines.
Cotton County, OK
Cottonwood County, MN
Covington County, AL
Covington County, MS
Covington city, VA
Coweta County, GA
Cowley County, KS
Cowlitz County, WA
Craig County, OK
Craig County, VA
Craighead County, AR
Crane County, TX
Craven County, NC
Crawford County, AR
Crawford County, GA
Crawford County, IA
Crawford County, IL
Crawford County, IN
Crawford County, KS
Crawford County, MI
Crawford County, MO
Crawford County, OH
Crawford County, PA
Crawford County, WI
Creek County, OK
Crenshaw County, AL
Crisp County, GA
Crittenden County, AR
Crittenden County, KY
Crockett County, TN
Crockett County, TX
Crook County, OR
Crook County, WY
Crosby County, TX
Cross County, AR
Crow Wing County, MN
Crowley County, CO
Culberson County, TX
Cullman County, AL
Culpeper County, VA
Cumberland County, IL
Cumberland County, KY
Cumberland County, ME
Cumberland County, NC
Cumberland County, NJ
Cumberland County, PA
Cumberland County, TN
Cumberland County, VA


### Exporting model results

Model results are exported and to be evaluated in a separate notebook.

In [13]:
df = pd.DataFrame(ls)
df.to_csv('model_rnn_3.csv', index = False) 
files.download('model_rnn_3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>